<a href="https://colab.research.google.com/github/RaginiMeshram/Tensorflow/blob/master/Copy_of_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [0]:
import h5py
import numpy as np
from PIL import Image
from matplotlib import  pyplot as plt
import os

In [7]:
os.getcwd()

'/content'

In [0]:
os.chdir('//content/drive/My Drive/Assignment_lane_detection/')

In [9]:
!pwd

/content/drive/My Drive/Assignment_lane_detection


In [11]:
os.listdir('/content/drive/My Drive/Assignment_lane_detection/')

['train.py',
 'labelHere.npy',
 'dataHere.npy',
 '__pycache__',
 'layers.py',
 'logs',
 'Segnet_model.py']

In [12]:
import os
import random
import sys
import warnings
import numpy as np
import pandas as pd
from itertools import chain
from skimage.io import imread, imshow, imread_collection, concatenate_images
from skimage.transform import resize
from skimage.morphology import label
from keras.utils import Progbar

from keras.models import Model, load_model
from keras.layers import Input
from keras.layers.core import Dropout, Lambda
from keras.layers.convolutional import Conv2D, Conv2DTranspose,Convolution2D
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras import backend as K

warnings.filterwarnings('ignore', category=UserWarning, module='skimage')

# Setting seed for reproducability
seed = 42
random.seed = seed
np.random.seed = seed
smooth = 1.
epochs = 100

Using TensorFlow backend.


In [0]:
train_data_path = '/content/drive/My Drive/Assignment_lane_detection/'
train_label_path = '/content/drive/My Drive/Assignment_lane_detection/'

In [0]:
from keras import callbacks  
# from keras.utils.visualize_util import plot
import keras.backend as K
K.set_image_data_format("channels_first")
from keras.layers import Convolution2D,Permute,Activation,Reshape,ZeroPadding2D
from keras.optimizers import RMSprop,SGD

#from Networks import Resnet_Modified
# import Resnet_Modified
import numpy as np
from keras.layers.convolutional import UpSampling2D

import tensorflow as tf
from keras.backend.tensorflow_backend import set_session




In [0]:
path2save = '/content/drive/My Drive/Assignment_lane_detection/'
log_dir = '/content/drive/My Drive/Assignment_lane_detection/log_unet/'


pretrainedPath = None
enableNorm = 0
imheight = 720
imwidth = 1280
depth = 3
classes = 2

In [0]:
data_shape = imheight*imwidth
classes = 2

train_data = np.load(train_data_path+'dataHere.npy')
# train_data  = train_data .astype("float32")
# train_data  = train_data/255.0

train_label = np.load(train_label_path+'labelHere.npy')
train_label = np.reshape(train_label,(len(train_label),data_shape,classes))

In [0]:
import skimage.io as io
import skimage.transform as trans
import numpy as np
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras import backend as keras
from keras.layers import Input,Conv2D,MaxPooling2D,AveragePooling2D,BatchNormalization,Dropout,Conv2DTranspose,concatenate,Reshape,Permute,Activation
from keras.models import Model

import sys
import warnings
import numpy as np
import pandas as pd
from itertools import chain
from skimage.io import imread, imshow, imread_collection, concatenate_images
from skimage.transform import resize
from skimage.morphology import label
from keras.utils import Progbar

from keras.models import Model, load_model
from keras.layers import Input
from keras.layers.core import Dropout, Lambda
from keras.layers.convolutional import Conv2D, Conv2DTranspose,Convolution2D
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras import backend as K

warnings.filterwarnings('ignore', category=UserWarning, module='skimage')

# Setting seed for reproducability
seed = 42
smooth = 1.
  
# Metric function
def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

# Loss funtion
def dice_coef_loss(y_true, y_pred):
    return -dice_coef(y_true, y_pred)


#Define the neural network imwidth,IMG_HEIGHT=imheight
def get_unet(n_ch=3,patch_height=imheight,patch_width=imwidth):
    inputs = Input(shape=(n_ch,patch_height,patch_width))
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same',data_format='channels_first')(inputs)
    conv1 = Dropout(0.2)(conv1)
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same',data_format='channels_first')(conv1)
    pool1 = MaxPooling2D((2, 2))(conv1)
    #
    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same',data_format='channels_first')(pool1)
    conv2 = Dropout(0.2)(conv2)
    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same',data_format='channels_first')(conv2)
    pool2 = MaxPooling2D((2, 2))(conv2)
    #
    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same',data_format='channels_first')(pool2)
    conv3 = Dropout(0.2)(conv3)
    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same',data_format='channels_first')(conv3)
    pool3 = MaxPooling2D((2, 2))(conv3)
    #
    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same',data_format='channels_first')(pool3)
    conv4 = Dropout(0.2)(conv4)
    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same',data_format='channels_first')(conv4)

    up = UpSampling2D(size=(2, 2))(conv4)
    up = concatenate([conv3,up],axis=1)
    conva = Conv2D(128, (3, 3), activation='relu', padding='same',data_format='channels_first')(up)
    conva = Dropout(0.2)(conva)
    conva = Conv2D(128, (3, 3), activation='relu', padding='same',data_format='channels_first')(conva)

    up1 = UpSampling2D(size=(2, 2))(conva)
    up1 = concatenate([conv2,up1],axis=1)
    conv4 = Conv2D(64, (3, 3), activation='relu', padding='same',data_format='channels_first')(up1)
    conv4 = Dropout(0.2)(conv4)
    conv4 = Conv2D(64, (3, 3), activation='relu', padding='same',data_format='channels_first')(conv4)
    #
    up2 = UpSampling2D(size=(2, 2))(conv4)
    up2 = concatenate([conv1,up2], axis=1)
    conv5 = Conv2D(32, (3, 3), activation='relu', padding='same',data_format='channels_first')(up2)
    conv5 = Dropout(0.2)(conv5)
    conv5 = Conv2D(32, (3, 3), activation='relu', padding='same',data_format='channels_first')(conv5)
    #
    conv6 = Conv2D(2, (1, 1), activation='relu',padding='same',data_format='channels_first')(conv5)
    conv6 = core.Reshape((2,patch_height*patch_width))(conv6)
    conv6 = core.Permute((2,1))(conv6)
    ############
    conv7 = core.Activation('softmax')(conv6)

    model = Model(input=inputs, output=conv7)

    # sgd = SGD(lr=0.01, decay=1e-6, momentum=0.3, nesterov=False)
    #model.compile(optimizer='sgd', loss='categorical_crossentropy',metrics=['accuracy'])
    model.compile(optimizer='sgd', loss='categorical_crossentropy',metrics=[dice_coef])

    return model

In [30]:
# get u_net model
net = get_unet()
net.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 3, 720, 1280) 0                                            
__________________________________________________________________________________________________
conv2d_35 (Conv2D)              (None, 32, 720, 1280 896         input_4[0][0]                    
__________________________________________________________________________________________________
dropout_17 (Dropout)            (None, 32, 720, 1280 0           conv2d_35[0][0]                  
__________________________________________________________________________________________________
conv2d_36 (Conv2D)              (None, 32, 720, 1280 9248        dropout_17[0][0]                 
____________________________________________________________________________________________

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:94: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("ac...)`


In [0]:
path2save1 = '/content/drive/My Drive/Assignment_lane_detection/Unet.h5'

In [0]:
modelCheck = callbacks.ModelCheckpoint(path2save1, monitor='loss', verbose=0, save_best_only=True, save_weights_only=True, mode='min')

TB_LOG = log_dir

In [0]:
tensorboard = callbacks.TensorBoard(log_dir=TB_LOG,
                                  histogram_freq=0,
                                  batch_size=8,
                                  write_graph=True,
                                  write_grads=False,
                                  write_images=False,
                                  embeddings_freq=0,
                                  embeddings_layer_names=None,
                                  embeddings_metadata=None)
# optimizer = SGD(lr=1e-15, momentum=0.9, decay=0.0, nesterov=False)
optimizer = RMSprop(lr=1e-5, rho=0.95, epsilon=1e-8, decay=0)

In [0]:
print ("Training the Model...")

# net.fit(train_data, train_label, batch_size = 1, epochs = 50, verbose=2,callbacks= [modelCheck],validation_split=0.2)
net.fit(train_data, train_label, batch_size= 4, epochs = 500, verbose=2,callbacks= [modelCheck,tensorboard])

path2save1 = '/content/drive/My Drive/Assignment_lane_detection/Unet.h5'
print ("Dumping Weights to file...")
net.save_weights(path2save1, overwrite=True)
print ("Models Saved :)")

Training the Model...
Epoch 1/500
 - 75s - loss: 0.1184 - dice_coef: 0.9472
Epoch 2/500
 - 75s - loss: 0.1169 - dice_coef: 0.9494
Epoch 3/500
 - 75s - loss: 0.1165 - dice_coef: 0.9495
Epoch 4/500
 - 75s - loss: 0.1162 - dice_coef: 0.9499
Epoch 5/500
 - 75s - loss: 0.1160 - dice_coef: 0.9499
Epoch 6/500
 - 75s - loss: 0.1158 - dice_coef: 0.9503
Epoch 7/500
 - 75s - loss: 0.1157 - dice_coef: 0.9499
Epoch 8/500
 - 75s - loss: 0.1156 - dice_coef: 0.9501
Epoch 9/500
 - 75s - loss: 0.1153 - dice_coef: 0.9502
Epoch 10/500
 - 75s - loss: 0.1152 - dice_coef: 0.9501
Epoch 11/500
 - 75s - loss: 0.1150 - dice_coef: 0.9504
Epoch 12/500
 - 75s - loss: 0.1149 - dice_coef: 0.9503
Epoch 13/500
 - 75s - loss: 0.1147 - dice_coef: 0.9504
Epoch 14/500
 - 75s - loss: 0.1146 - dice_coef: 0.9504
Epoch 15/500
 - 75s - loss: 0.1143 - dice_coef: 0.9505
Epoch 16/500
 - 75s - loss: 0.1136 - dice_coef: 0.9505
Epoch 17/500
 - 75s - loss: 0.1126 - dice_coef: 0.9509
Epoch 18/500
 - 75s - loss: 0.1113 - dice_coef: 0.95